
---

# 🔐 **Authentication & Authorization in FastAPI**

> FastAPI supports secure authentication methods like **OAuth2**, **JWT**, and **API keys**, and even lets you implement **role-based access control (RBAC)** with minimal code.

---

## 🔑 OAuth2 Login Flow (Basic)

📌 Best for: **Username/password login** using form input (e.g., HTML login form or `/docs` UI).

```python
from fastapi import FastAPI, Depends, HTTPException
from fastapi.security import OAuth2PasswordBearer, OAuth2PasswordRequestForm

app = FastAPI()

# Setup: Accept tokens via "Authorization: Bearer <token>"
oauth2_scheme = OAuth2PasswordBearer(tokenUrl="token")

# Step 1: Login endpoint returns token
@app.post("/token")
def login(form_data: OAuth2PasswordRequestForm = Depends()):
    if form_data.username == "admin" and form_data.password == "1234":
        return {"access_token": "mytoken", "token_type": "bearer"}
    raise HTTPException(status_code=401, detail="Invalid credentials")

# Step 2: Secure route that uses token
@app.get("/profile")
def get_profile(token: str = Depends(oauth2_scheme)):
    return {"token_used": token}
```

✅ `/docs` UI includes a working login form (using `form-data`).

---

## 🔐 JWT Authentication (Secure, Stateless)

📌 Best for: **Production-grade APIs** with stateless token-based security.

### ✅ Install `pyjwt`

```bash
pip install pyjwt
```

### 🔧 JWT Helpers

```python
import jwt
from datetime import datetime, timedelta
from fastapi import HTTPException

SECRET = "mysecretkey"

# Create a JWT token
def create_token(data: dict):
    to_encode = data.copy()
    to_encode["exp"] = datetime.utcnow() + timedelta(minutes=30)
    return jwt.encode(to_encode, SECRET, algorithm="HS256")

# Decode and verify the token
def decode_token(token: str):
    try:
        return jwt.decode(token, SECRET, algorithms=["HS256"])
    except jwt.ExpiredSignatureError:
        raise HTTPException(status_code=401, detail="Token expired")
    except jwt.InvalidTokenError:
        raise HTTPException(status_code=403, detail="Invalid token")
```

### 🔒 Login + Secure Route using JWT

```python
@app.post("/login")
def login_user(form_data: OAuth2PasswordRequestForm = Depends()):
    if form_data.username == "admin" and form_data.password == "1234":
        token = create_token({"sub": form_data.username, "role": "admin"})
        return {"access_token": token, "token_type": "bearer"}
    raise HTTPException(status_code=401, detail="Invalid credentials")

@app.get("/secure")
def secure_info(token: str = Depends(oauth2_scheme)):
    data = decode_token(token)
    return {"user": data.get("sub"), "role": data.get("role")}
```

---

## 🧪 API Key in Header (Quick & Simple)

📌 Best for: **Internal APIs**, automation scripts, cron jobs, etc.

```python
from fastapi import Header, HTTPException

API_KEY = "supersecretkey"

@app.get("/internal")
def internal_route(x_api_key: str = Header(...)):
    if x_api_key != API_KEY:
        raise HTTPException(status_code=403, detail="Invalid API Key")
    return {"status": "Authorized"}
```

🔐 Just pass this header:

```
X-API-Key: supersecretkey
```

---

## 🧑‍⚖️ Role-Based Access Control (RBAC)

📌 Best for: **Admin/user separation** — restrict who can access what.

```python
# Use decoded JWT to extract role
def get_user_role(token: str = Depends(oauth2_scheme)):
    payload = decode_token(token)
    return payload.get("role")

# Protect route to allow only admins
@app.get("/admin")
def admin_panel(role: str = Depends(get_user_role)):
    if role != "admin":
        raise HTTPException(status_code=403, detail="Admins only")
    return {"status": "Welcome Admin"}
```

✅ Make sure to include `"role": "admin"` in the JWT payload when creating it.

---

## ✅ Summary Table

| 🔹 Feature             | 🔍 What It Does              | 💼 Best Use Case      |
| ---------------------- | ---------------------------- | --------------------- |
| `OAuth2PasswordBearer` | Username/password login      | Basic login form      |
| JWT with `pyjwt`       | Secure, stateless token auth | Production APIs       |
| API Key in Header      | Simple header-based access   | Internal tools        |
| Role-Based Protection  | Limit access by user roles   | Admin/user dashboards |

---